Importing Required Libraries

In [1]:
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Downloading and Importing Preprocessing Function

In [4]:
!wget -O Prepare_supervised_data_functions.py "https://raw.githubusercontent.com/orifelszer/CrimeData/refs/heads/main/Prepare_supervised_data_functions.py"
from Prepare_supervised_data_functions import Preprocessing

--2025-01-11 13:28:09--  https://raw.githubusercontent.com/orifelszer/CrimeData/refs/heads/main/Prepare_supervised_data_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8311 (8.1K) [text/plain]
Saving to: ‘Prepare_supervised_data_functions.py’

Prepare_supervised_ 100%[===================>]   8.12K  --.-KB/s    in 0s      

2025-01-11 13:28:09 (60.3 MB/s) - ‘Prepare_supervised_data_functions.py’ saved [8311/8311]



Defining Paths for Crime Datasets (2019–2024)


In [5]:
# === Downloading and Extracting Crime Data Files ===
import os
import zipfile

!git clone https://github.com/orifelszer/CrimeData.git

zip_folder = 'CrimeData'
# Extracting only ZIP files for the years 2019 to 2024
zip_files = [f for f in os.listdir(zip_folder) if f.startswith('crimes') and any(str(year) in f for year in range(2019, 2025))]

# Unzipping files and saving the new folder paths
extracted_folders = []

for zip_file in zip_files:
    zip_path = os.path.join(zip_folder, zip_file)
    extract_path = os.path.join(zip_folder, zip_file.replace('.zip', ''))
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    extracted_folders.append(extract_path)
    print(f"Extracted: {zip_file} -> {extract_path}")


Cloning into 'CrimeData'...
remote: Enumerating objects: 712, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 712 (delta 182), reused 96 (delta 96), pack-reused 477 (from 3)
Receiving objects: 100% (712/712), 228.16 MiB | 31.44 MiB/s, done.
Resolving deltas: 100% (365/365), done.
Extracted: crimes2020.zip -> CrimeData/crimes2020
Extracted: crimes2023.zip -> CrimeData/crimes2023
Extracted: crimes2024.zip -> CrimeData/crimes2024
Extracted: crimes2019.zip -> CrimeData/crimes2019
Extracted: crimes2021.zip -> CrimeData/crimes2021
Extracted: crimes2022.zip -> CrimeData/crimes2022


Loading Crime Data into Pandas DataFrames

In [6]:
csv_files = []
for folder in extracted_folders:
    csv_files += glob.glob(os.path.join(folder, "*.csv"))

dataframes = {f"Crimes_{file[-8:-4]}": pd.read_csv(file) for file in csv_files}
combined_data = pd.concat(dataframes.values(), axis=0, ignore_index=True)

Removing Invalid Rows

In [7]:
# Rows where 'StatisticGroupKod' equals -1 are removed, as -1 is a typing mistake in the data.
combined_data = combined_data[combined_data['StatisticGroupKod'] != -1]

Splitting Features and Target Variable

In [8]:
features = combined_data.drop(['StatisticGroup', 'StatisticGroupKod'], axis=1)
target = combined_data['StatisticGroup']

# Splitting Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(
    features,
    target,
    test_size=0.2,
    random_state=42,
    stratify=target
    )

Applying Preprocessing Function

In [9]:
X_train_cleaned, train_mappings, scaler, label_encoders = Preprocessing(X_train, fit_scaler=True)
X_test_cleaned, _, _, _ = Preprocessing(X_test, train_mappings=train_mappings, scaler=scaler)

/content/Prepare_supervised_data_functions.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(fill_statistic_area_random(x)))
/content/Prepare_supervised_data_functions.py:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(fill_statistic_area_random(x)))


Retaining Relevant Columns and Aligning Datasets

In [10]:
# Retaining only columns where the sum of values in the training set is significant (at least 10)
columns_to_keep = [col for col in X_train_cleaned.columns if X_train_cleaned[col].sum() >= 10]
X_train_cleaned = X_train_cleaned[columns_to_keep]

# Ensuring the test set includes the same columns as the training set
for col in columns_to_keep:
    if col not in X_test_cleaned.columns:
        X_test_cleaned[col] = 0
X_test_cleaned = X_test_cleaned[X_train_cleaned.columns]

Aligning Target Variables with Cleaned Datasets

In [11]:
y_train = y_train.loc[X_train_cleaned.index]
y_test = y_test.loc[X_test_cleaned.index]

Encoding Target Variable

In [12]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

Saving Cleaned Datasets and Encoded Targets as CSV Files

In [13]:
X_train_cleaned.to_csv('X_train_supervised.csv', index=False)
X_test_cleaned.to_csv('X_test_supervised.csv', index=False)
pd.DataFrame(y_train_encoded).to_csv('y_train_supervised.csv', index=False, header=['target'])
pd.DataFrame(y_test_encoded).to_csv('y_test_supervised.csv', index=False, header=['target'])